In [1]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
from scraper import *
import numpy as np
import scipy.sparse as sp
from tables import *
%load_ext autoreload
%autoreload 2

# display all pandas columns

pd.set_option('display.max_columns', 100)

# Tables used for D3 visualisation 
Tables:
* author
* cosigner
* interest

In [2]:
tables = Tables()

/Users/joachimmuth/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (10,27) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
interest = tables.interest()
interest.head()

TagName,MemberCouncilNumber,Politique d‘Etat,Politique internationale,Politique de sécurité,Politique européenne,Droit,Économie,Finances,Questions sociales,Éducation,Médias et communication,Science et recherche,Emploi et travail,Transports,Environnement,Agriculture,Énergie,Parlement,Droit civil,Droit pénal,Justice,Droit international,Droits de l'homme,Fiscalité,Politique migratoire,Culture,Protection sociale,Santé,Aménagement du territoire et logement
0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,17,25,14,4,16,22,7,20,5,6,2,0,19,24,9,11,7,0,0,0,0,0,0,12,6,0,16,4
2,11,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,12,5,3,2,3,3,10,10,4,5,3,4,0,8,4,3,1,1,0,0,0,0,0,0,3,1,0,7,3
4,14,20,12,21,5,18,53,34,23,18,18,10,0,82,29,23,9,4,0,0,0,0,0,0,10,10,0,26,8


In [ ]:
authors = tables.author()
authors.head()

In [ ]:
cosigners = tables.cosigner()
cosigners

## Explanation of the Table

* **Business** is the table of initiatives
* **BusinessRole** table of all the person who act for or against a business. IMPORTANT! **BusinessRole.MemberCouncilNumber** link to **MemberCouncil.PersonNumber**. **BusinessShortNumber** link businesses and roles.
    * Auteur
    * Cosignataire
    * Porte-parole
    * Opposant(e)
    * Repreneur
    
* **BusinessType** contain all 18 types of businesses. As the type is reapeated in **Business.BusinessType** and **Business.BusinessTypeName**, it is not useful for us.
* **BusinessStatus** give the current status of a business. Again, this information is given in **Business.BusinessStatusText**, so not useful


* Tag are stored in "TagNames" Array
* **Tags** are easier (as it is integer) 

# Compiling the _Cosignataire_ info into an adjacency matrix for d3

In [ ]:
adj = pd.read_csv('data/adj.csv')

In [ ]:
adj.rename(columns={'PersonNumber': 'PersonIdCode'}, inplace=True)

In [ ]:
adj.set_index('PersonIdCode', inplace=True)

In [ ]:
adj.head(2)

In [ ]:
people = pd.read_csv('data/people_jonas.csv').set_index('PersonIdCode')

In [ ]:
people.loc[2023].PersonNumber

In [ ]:
def fill_adj(adj, people):
    # getting a list of active members (we're only interested in them)
    active = people.PersonNumber.tolist()
    # going through an empty adj matrix with PersonIdCodes as rows and columns
    for row in adj.iterrows():
        person_id = row[0]
        # converting from PersonIdCode to PersonNumber for friends search
        person_number = people.loc[person_id].PersonNumber
        # searching co-sign friends w/ the function defined above
        friends_matrix = get_friends(friends, person_number)
        # looping through friends to fill the matrix
        for friend in friends_matrix.iterrows():
            # checking if active
            if friend[1].Person_num in active:
                # converting from PersonNumber to PersonIdCode
                friend_id = people.loc[people.PersonNumber == friend[1].Person_num].index.tolist()[0]
                # Updating matrix
                adj.loc[person_id, str(friend_id)] = friend[1].times_cosigner

In [ ]:
fill_adj(adj, people)

## WARNING: the adjacency matrix is **NOT** symmetric

For now, I'll be only considering rows in the viz, but might make sense to consider columns, dunno.

In [ ]:
adj.head(2)

In [ ]:
adj.to_csv('data/adj_cosign.csv')

In [ ]:
adj.to_json('data/adj_cosign.json', orient='index')